In [ ]:
import torch
import numpy as np

import utils

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# log file
model_id = 8
title = 'arl50'
log = utils.Logger(verbose=True, title=title)
log.logger.info("1-1 ARL50-{}".format(model_id))

## Data Preparation

In [ ]:
# path of img data
pth_train = 'Data/ISIC2017/Aug_Training_Data'
pth_valid = 'Data/ISIC2017/ISIC-2017_Validation_Data'
pth_test = 'Data/ISIC2017/ISIC-2017_Test_Data'


ann_train = utils.Annotation('Data/ISIC2017/ISIC-2017_Training_Aug_Part3_GroundTruth.csv')
ann_valid = utils.Annotation('Data/ISIC2017/ISIC-2017_Validation_Part3_GroundTruth.csv')

In [ ]:
from torchvision import transforms
from torch.utils import data

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])
                                ])

# augmentation transform
transform_train = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.2), ratio=(3/4, 4/3)),
                                      transforms.RandomAffine(10, scale=(0.9, 1.1)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                      ])

In [ ]:
# create train dataset
train_data = utils.RandomPatch(ann_train.df, pth_train, transform=transform_train)
train_loader = data.DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True, num_workers=2)

# create validation dataset
valid_data = utils.RandomPatch(ann_valid.df, pth_valid, transform=transform)
valid_loader = data.DataLoader(valid_data, batch_size=32, shuffle=False, num_workers=2)

## Network Design

In [ ]:
# GPU setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import nets

model = nets.arlnet50(pretrained=True, num_classes=3)
# 1-2
# model = nets.resnet50(pretrained=True, num_classes=3)
# # 2
# model = nets.arlnet18(pretrained=False, num_classes=3)

# model = utils.load_model(device, name="arl50.pkl")
# model = utils.load_state_dict(model, device, name="arl50_dict.pth")
model.to(device)

## Train
### Criterion

Focal Loss
$$
{\text{FL}(p_{t}) = - \alpha_t (1 - p_{t})^\gamma \log\left(p_{t}\right)} 
$$

In [ ]:
# setting weights
# # 2-1/3
# class_weight = [1 for _ in range(ann_train.class_num)]
# 2-2/4
# class_nums = ann_train.count_samples()
# class_weight = [len(ann_train.df)/(len(class_nums)*num) for num in class_nums]
class_weight = [2.5, 2.5, 0.5]

# 2-1/2
criterion = utils.FocalLoss(alpha=class_weight, gamma=2, num_classes=ann_train.class_num, reduction='sum')
# # 2-3/4
# criterion = nn.CrossEntropyLoss(weight=torch.Tensor(class_weight).to(device), reduction='sum')

log.logger.info(criterion)

In [ ]:
# initial test
eval_metrics = utils.Evaluation(device, ann_train.categories, 0)
eval_metrics.get_probs(model, valid_loader)
eval_metrics.complete_scores()
log.logger.info("Initial Test: valid_acc = {:.4f}, valid_bacc = {:.4f}, f1_score = {}, macro_auc = {}".format(eval_metrics.acc, eval_metrics.b_acc, eval_metrics.f1_score, eval_metrics.roc_auc['macro']))

### Train

The mini-batch SGD algorithm with a batch size of 32 was adopted as the optimizer. The learning rate was initialized to
0.01 for training ARL-CNN14 from scratch and 0.0001 for fine-tuning ARL-CNN50 with pre-trained parameters, and was
reduced by half very 30 epochs. The initial weighting factor of the attention feature maps was set to 0.001 in each ARL block
when fine-tuning the ARL-CNN50. The maximum epoch number was set to 100.

We used the officially provided validation set to monitor the performance of our model and stopped the training process when the network fell into overfitting.

In [ ]:
from torch import optim

In [ ]:
def train(model, train_loader, test_loader, max_epoch=100, test_period=5, early_threshold=5):
    """ train with a scheduler on learning rate

    Args:
        test_period (int): period of test
        early_threshold (int): threshold for early stoppig strategy, which pays attention to acc on test set
    """
    N_train = len(train_loader.dataset)
    patience = early_threshold
    
    # make sure the model is in the training mode
    model.train()

    global epoch
    for epoch in range(epoch, max_epoch):
        cost = 0
        correct = 0
        
        for x, y in train_loader:
            # setting GPU
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            
            cost += loss.item()
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y).sum().item()
            
        cost = cost / N_train
        costs.append(cost)    # average cost
        

        # ! acc on train in train mode
        acc = correct / N_train
        train_accs.append(acc)

        # adjsut learning rate
        scheduler.step()


        if epoch % test_period == 0:
            eval_metrics.get_probs(model, test_loader)
            eval_metrics.complete_scores()
            test_accs.append(eval_metrics.acc)
            b_accs.append(eval_metrics.b_acc)
            f1_scores.append(eval_metrics.f1_score)
            auc = eval_metrics.roc_auc['macro']
            auces.append(auc)
            
            if auc > eval_metrics.best_score:
                eval_metrics.best_score = auc
                patience = early_threshold
                # 在测试集上准确率上升时，保存模型参数
                utils.save_state_dict(model, name="{}_dict_{}.pth".format(title, model_id))
            else:
                patience -= 1
                if patience == 0:
                    break

            # (每个epoch）保存checkpoint
            utils.check_train(log, model, optimizer, epoch, scheduler, pth_check='ch_train_{}.pth'.format(title))
            utils.check_eval(log, costs, train_accs, test_accs, b_accs, f1_scores, auces, pth_check='ch_eval_{}.pth'.format(title), verbose=False)
                
            log.logger.info("{:3d} cost: {:.4f}\ttrain_acc: {:.4f}\ttest_acc: {:.4f}\ttest_bacc: {:.4f}\tf1_score: {}\tauc: {:.4f}".format(
                epoch, cost, acc, test_accs[-1], b_accs[-1], f1_scores[-1], auces[-1]))

            # change back to training mode    
            model.train()

In [ ]:
# setting
## filename
model_file = '{}-{}.pkl'.format(title, model_id)

## hyper-params
init_lr = 1e-4
weight_decay = 1e-2
max_epoch = 100
test_period = 1
early_threshold = 20

## optimizer: https://pytorch.org/docs/stable/optim.html
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=init_lr, momentum=momentum, weight_decay=weight_decay)


## learning rate decay
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

# 初始化训练状态
epoch = utils.load_train(log, model, optimizer, scheduler)
costs, train_accs, test_accs, b_accs, f1_scores, auces = utils.load_eval(log)

In [ ]:
# # load from checkpoint 从checkpoint加载继续训练
# epoch = utils.load_train(log, model, optimizer, scheduler, pth_check='ch_train_{}.pth'.format(title))
# costs, train_accs, test_accs, b_accs, f1_scores, auces = utils.load_eval(log, pth_check='ch_eval_{}.pth'.format(title))

log.logger.info("Training...\n{}\n{}".format(optimizer, scheduler))
train(model, train_loader, valid_loader, max_epoch, test_period, early_threshold)

In [ ]:
# # save checkpoint
# utils.check_train(log, model, optimizer, epoch, scheduler)
# utils.check_eval(log, costs, train_accs, test_accs, b_accs, f1_scores, auces)

In [ ]:
# save model
utils.save_model(model, name=model_file)
log.logger.info("Filename: {}\ncosts = {}\ntrain_accs = {}\ntest_acc = {}\nb_accs = {}\nf1_scores = {}\nauces = {}".format(
    model_file, costs, train_accs, test_accs, b_accs, f1_scores, auces))

## Evaluation

Implemented in [Evaluation.ipynb](Evaluation.ipynb)

In the test stage, we used the same patch extraction method to randomly crop nine patches from each test image, fed them to the trained network, and averaged the obtained scores as the predicted score of the image.

In [ ]:
# create test dataset
del train_loader, valid_loader

ann_test = utils.Annotation('Data/ISIC2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv')
test_data = utils.RandomPatch(ann_test.df, pth_test, transform=transform)
test_loader = data.DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
eval_metrics = utils.Evaluation(device, ann_test.categories)


def evaluation_report(model, data_loader):
    y, prob = eval_metrics.get_probs(model, data_loader)
    probs = [prob]
    for _ in range(8):
        _, prob = eval_metrics.get_probs(model, data_loader)
        probs.append(prob)
    prob = np.mean(probs, axis=0)
    eval_metrics.prob = prob
    
    eval_metrics.complete_scores()
    
    log.logger.info("Evaluation on test set\n{}".format(eval_metrics.report))
    print("MEL Acc: {:.4f}".format(eval_metrics.mel_acc))
    print("SK Acc: {:.4f}".format(eval_metrics.sk_acc))
    roc_auc = eval_metrics.roc_auc
    
    for key in roc_auc.keys():
        log.logger.info("AUC({}): {}".format(key, roc_auc[key]))

In [ ]:
evaluation_report(model, test_loader)